In [56]:
import requests
import pandas as pd
import numpy as np
import time

In [4]:
url = 'https://api.pushshift.io/reddit/search/submission'

## Run this first to get the last utc time for the first run

In [45]:
params = {
    'subreddit': 'Houston',
    'size': 100    
}

res = requests.get(url, params)

res.status_code

200

In [46]:
res.json().keys()

dict_keys(['data'])

In [47]:
posts = res.json()['data']

In [48]:
posts[0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'swordpriest1',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_anh5j',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1630951521,
 'domain': 'self.houston',
 'full_link': 'https://www.reddit.com/r/houston/comments/pj4tvc/want_to_build_a_stand_alone_apartment_in_backyard/',
 'gildings': {},
 'id': 'pj4tvc',
 'is_created_from_ads_ui': False,
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comme

In [49]:
df = pd.DataFrame(posts)[['subreddit', 'title', 'selftext', 'created_utc']]
df.tail()

,subreddit,title,selftext,created_utc
95,houston,A legit question.,,1630805160
96,houston,Sweet pup rescued from a hot car is up for ado...,,1630803508
97,houston,Beltway toll “cash” lane,[removed],1630801894
98,houston,Tiki Macaw Buzz Fest 2021,,1630796555
99,houston,Tiki Macaw Buzz Fest 2021 Part 1,,1630796545


## Now run the scraper function

In [50]:
#Will scrape redit/Houston for n times and return a df
#will need to run it one time outside of function to get the laststatus time and create the df
last_time = df['created_utc'].iloc[-1]
#1630796537

def scraped(n):
    df_final = df
    for i in range (n):
        
        params = {
        'subreddit': 'Houston',
        'size': 100,
        'before': df['created_utc'].iloc[-1]
        }
        res = requests.get(url, params)
        posts = res.json()['data']
        df2 = pd.DataFrame(posts)[['subreddit', 'title', 'selftext', 'created_utc']]
        df_final = pd.concat([df_final, df2])
        #need to update last_time
        #last_time = df['created_utc'].iloc[-1]
        time.sleep(10)
    
    return df_final


## Define number of iterations, remember you pull 100 post per iteration

In [107]:
#Call the function
df_Houston = scraped(150)

## Quick test of intergrity of data frame to make sure data aquisition is complete

In [108]:
df_Houston.head()

,subreddit,title,selftext,created_utc
0,houston,Want to build a stand alone apartment in backyard,Hello everyone. I want to build a stand alone ...,1630951521
1,houston,Relocating from Lex KY,Hi! Considering relocating from Lexington KY a...,1630948138
2,houston,Anywhere in Houston that can mount bindings to...,Just received some warranty replacement skis f...,1630943989
3,houston,Anyone else out of power? I'm in energy corridor.,[removed],1630943642
4,houston,Places to Study,Hi everyone! I just started school and I’m loo...,1630943452


In [109]:
df_Houston.shape

(15100, 4)

In [110]:
df_Houston['selftext'].value_counts()

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             7387
[removed]                                                                                                                                                                             

In [111]:
len(df_Houston.loc[df_Houston['selftext'] == '[removed]'])

1669

## Remove blanks and [removed] content

In [112]:
df_Houston['selftext'].replace('', np.nan, inplace=True)

In [113]:
df_Houston['selftext'].replace('[removed]', np.nan, inplace=True)

In [114]:
df_Houston.isnull().sum()

subreddit         0
title             0
selftext       9056
created_utc       0
dtype: int64

In [115]:
df_Houston.dropna(inplace = True)

In [116]:
df_Houston.shape

(6044, 4)

## Write to a csv

In [117]:
df_Houston.to_csv('data/df_Houston.csv', index=False)